In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/컴퓨터 비전/computer-vision

Mounted at /content/drive/
/content/drive/MyDrive/컴퓨터 비전/computer-vision


In [2]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataloader.dataset import CustomDataset
from model.unet import UNet
from tool.encode import rle_encode
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2

SAVE_ROOT = "/content/drive/MyDrive/컴퓨터 비전/computer-vision/model/checkpoint"
DATA_ROOT = "/content/drive/MyDrive/컴퓨터 비전/data"

def set_dataloader():
    def set_tta():
        transform = A.Compose(
          [
              A.Resize(224, 224),
              A.Normalize(),
              ToTensorV2()
          ])
        return transform
    test_dataset = CustomDataset(csv_file='./test.csv', transform=set_tta(), infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
    return test_dataloader

def set_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet().to(device)
    return model, device

def train():
  model, device = set_model()
  train_dataloader = set_dataloader()
  model_name = "unet"

    # loss function과 optimizer 정의
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  # training loop
  for epoch in range(20):  # 20 에폭 동안 학습합니다.
      model.train()
      epoch_loss = 0
      for images, masks in tqdm(train_dataloader):
          images = images.float().to(device)
          masks = masks.long().to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, masks.squeeze(1))
          loss.backward()
          optimizer.step()

          epoch_loss += loss.item()

      print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(train_dataloader)}')
      torch.save(model, os.path.join(SAVE_ROOT, model_name+"_epoch"+epoch+".pt"))

train()

FileNotFoundError: [Errno 2] No such file or directory: './test.csv'